# Import & Setup

In [218]:
!pip install pingouin
!pip install scikit_posthocs
import scikit_posthocs as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import pingouin as pg
import calendar
from datetime import datetime
pd.set_option('display.max_columns', None)

pio.templates.default = "ggplot2"

In [219]:
from google.colab import drive
drive.mount('/content/gdrive')

# directory
%cd '/content/gdrive/My Drive/Oxford/ML_for_Social_Good'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Oxford/ML_for_Social_Good


In [220]:
# import
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

train["Set"] = "train"
test["Set"] = "test"

df = pd.concat([train, test])
print(df.shape)

(5160, 45)


In [221]:
df.columns

Index(['ID', 'District', 'Block', 'CultLand', 'CropCultLand',
       'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth',
       'CropEstMethod', 'RcNursEstDate', 'SeedingSowingTransplanting',
       'SeedlingsPerPit', 'NursDetFactor', 'TransDetFactor',
       'TransplantingIrrigationHours', 'TransplantingIrrigationSource',
       'TransplantingIrrigationPowerSource', 'TransIrriCost', 'StandingWater',
       'OrgFertilizers', 'Ganaura', 'CropOrgFYM', 'PCropSolidOrgFertAppMethod',
       'NoFertilizerAppln', 'CropbasalFerts', 'BasalDAP', 'BasalUrea',
       'MineralFertAppMethod', 'FirstTopDressFert', '1tdUrea', '1appDaysUrea',
       '2tdUrea', '2appDaysUrea', 'MineralFertAppMethod.1', 'Harv_method',
       'Harv_date', 'Harv_hand_rent', 'Threshing_date', 'Threshing_method',
       'Residue_length', 'Residue_perc', 'Stubble_use', 'Acre', 'Yield',
       'Set'],
      dtype='object')

In [222]:
datetime_cols = ["CropTillageDate","RcNursEstDate","SeedingSowingTransplanting","Harv_date","Threshing_date"]

for col in datetime_cols:
  df[col] = pd.to_datetime(df[col])#.dt.date

In [223]:
df.dtypes

ID                                            object
District                                      object
Block                                         object
CultLand                                       int64
CropCultLand                                   int64
LandPreparationMethod                         object
CropTillageDate                       datetime64[ns]
CropTillageDepth                               int64
CropEstMethod                                 object
RcNursEstDate                         datetime64[ns]
SeedingSowingTransplanting            datetime64[ns]
SeedlingsPerPit                              float64
NursDetFactor                                 object
TransDetFactor                                object
TransplantingIrrigationHours                 float64
TransplantingIrrigationSource                 object
TransplantingIrrigationPowerSource            object
TransIrriCost                                float64
StandingWater                                f

In [224]:
'''
cat_col = ["District","Block","CropTillageDate","CropTillageDepth","CropEstMethod","RcNursEstDate","SeedingSowingTransplanting","TransplantingIrrigationSource","TransplantingIrrigationPowerSource",
           "PCropSolidOrgFertAppMethod","NoFertilizerAppln","CropbasalFerts","MineralFertAppMethod.1","Harv_method","Harv_date","Threshing_date","Threshing_method","Stubble_use"
           ]
num_col = ["CultLand","CropCultLand","SeedlingsPerPit","TransplantingIrrigationHours","TransIrriCost","StandingWater","Ganaura","CropOrgFYM",
           "BasalDAP","BasalUrea","1tdUrea","1appDaysUrea","2tdUrea","2appDaysUrea","Harv_hand_rent","Residue_length","Residue_perc","Acre","Yield"
           ]

for col in cat_col:
  fig = px.histogram(df, x=col, width=650,height=300)
  fig.show()

for col in num_col:
  fig = px.box(df, x=col, width=650,height=300)
  fig.show()

'''
# Visualizations are all in the google sheets

'\ncat_col = ["District","Block","CropTillageDate","CropTillageDepth","CropEstMethod","RcNursEstDate","SeedingSowingTransplanting","TransplantingIrrigationSource","TransplantingIrrigationPowerSource",\n           "PCropSolidOrgFertAppMethod","NoFertilizerAppln","CropbasalFerts","MineralFertAppMethod.1","Harv_method","Harv_date","Threshing_date","Threshing_method","Stubble_use"\n           ]\nnum_col = ["CultLand","CropCultLand","SeedlingsPerPit","TransplantingIrrigationHours","TransIrriCost","StandingWater","Ganaura","CropOrgFYM",\n           "BasalDAP","BasalUrea","1tdUrea","1appDaysUrea","2tdUrea","2appDaysUrea","Harv_hand_rent","Residue_length","Residue_perc","Acre","Yield"\n           ]\n\nfor col in cat_col:\n  fig = px.histogram(df, x=col, width=650,height=300)\n  fig.show()\n\nfor col in num_col:\n  fig = px.box(df, x=col, width=650,height=300)\n  fig.show()\n\n'

# Data cleaning

In [225]:
# One row has Jamui as district but Gurua as Block, which is a Gaya block -- correcting its District
df.loc[(df["District"]=="Jamui") & (df["Block"]=="Gurua")].index
df.loc[2177,"District"] = "Gaya"

In [226]:
# Creating a yield-per-acre column
df["Yield_per_Acre"] = df["Yield"] / df["Acre"]

In [227]:
px.box(df, x="Yield_per_Acre",width=650, height=350)

# Correlation analysis

In [228]:
corrmap = df.rcorr(stars=True).round(2)
corrmap.to_csv('correlation_map.csv')
df.rcorr(stars=False)
# note: upper triangle is p-values

/usr/local/lib/python3.10/dist-packages/pingouin/correlation.py:1082: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

/usr/local/lib/python3.10/dist-packages/pingouin/correlation.py:1087: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.

/usr/local/lib/python3.10/dist-packages/pingouin/correlation.py:1082: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numer

,CultLand,CropCultLand,CropTillageDepth,SeedlingsPerPit,TransplantingIrrigationHours,TransIrriCost,StandingWater,Ganaura,CropOrgFYM,NoFertilizerAppln,BasalDAP,BasalUrea,1tdUrea,1appDaysUrea,2tdUrea,2appDaysUrea,Harv_hand_rent,Residue_length,Residue_perc,Acre,Yield,Yield_per_Acre
CultLand,-,0.,0.006,0.000,0.223,0.185,0.000,0.766,0.889,0.001,0.000,0.000,0.000,0.000,0.000,0.044,0.000,0.000,0.000,0.000,0.000,0.17
CropCultLand,0.967,-,0.000,0.001,0.473,0.699,0.000,0.437,0.064,0.000,0.000,0.000,0.000,0.000,0.000,0.045,0.000,0.000,0.000,0.000,0.000,0.058
CropTillageDepth,0.038,0.05,-,0.000,0.284,0.000,0.000,0.000,0.000,0.000,0.687,0.013,0.126,0.000,0.000,0.356,0.184,0.207,0.000,0.703,0.854,0.571
SeedlingsPerPit,-0.053,-0.046,-0.053,-,0.796,0.006,0.421,0.334,0.003,0.004,0.739,0.001,0.028,0.099,0.047,0.514,0.090,0.024,0.155,0.016,0.159,0.542
TransplantingIrrigationHours,0.017,0.01,0.015,-0.004,-,0.000,0.000,0.643,0.202,0.002,0.464,0.02,0.545,0.527,0.863,0.000,0.375,0.001,0.701,0.119,0.638,0.001
TransIrriCost,-0.021,-0.006,-0.164,0.044,0.113,-,0.000,0.000,0.000,0.009,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.092,0.000,0.000,0.000
StandingWater,0.176,0.208,-0.123,-0.012,-0.068,-0.167,-,0.000,0.03,0.000,0.665,0.001,0.000,0.000,0.637,0.000,0.729,0.000,0.260,0.013,0.995,0.615
Ganaura,0.007,0.018,0.179,-0.022,0.011,-0.122,-0.128,-,0.000,0.002,0.000,0.000,0.001,0.000,0.514,0.488,0.392,0.000,0.629,0.000,0.126,0.620
CropOrgFYM,0.003,0.046,0.225,0.075,0.032,0.652,-0.054,0.492,-,0.000,0.000,0.000,0.000,0.000,0.000,0.763,0.000,0.001,0.358,0.000,0.000,0.303
NoFertilizerAppln,-0.047,-0.071,-0.118,-0.041,0.044,0.041,-0.266,-0.07,-0.235,-,0.000,0.000,0.000,0.000,0.707,0.613,0.562,0.000,0.052,0.565,0.198,0.000


In [229]:
# heatmap
fig = px.imshow(corrmap,color_continuous_scale=px.colors.sequential.RdBu,width=900,height=900)
fig.update_yaxes(nticks=40)
fig.update_xaxes(nticks=40, tickangle=45)
fig.show()

# EDA

In [230]:
print(df.Acre.min())
print(df.Acre.max())

0.0454545454545454
2.1875


In [231]:
# Mapping Districts to Blocks
df.loc[df["District"]=="Jamui"].Block.value_counts()
# -> put in VariableDescription sheets

Jamui     840
Khaira    669
Lohra       1
Name: Block, dtype: int64

In [232]:
df.columns

Index(['ID', 'District', 'Block', 'CultLand', 'CropCultLand',
       'LandPreparationMethod', 'CropTillageDate', 'CropTillageDepth',
       'CropEstMethod', 'RcNursEstDate', 'SeedingSowingTransplanting',
       'SeedlingsPerPit', 'NursDetFactor', 'TransDetFactor',
       'TransplantingIrrigationHours', 'TransplantingIrrigationSource',
       'TransplantingIrrigationPowerSource', 'TransIrriCost', 'StandingWater',
       'OrgFertilizers', 'Ganaura', 'CropOrgFYM', 'PCropSolidOrgFertAppMethod',
       'NoFertilizerAppln', 'CropbasalFerts', 'BasalDAP', 'BasalUrea',
       'MineralFertAppMethod', 'FirstTopDressFert', '1tdUrea', '1appDaysUrea',
       '2tdUrea', '2appDaysUrea', 'MineralFertAppMethod.1', 'Harv_method',
       'Harv_date', 'Harv_hand_rent', 'Threshing_date', 'Threshing_method',
       'Residue_length', 'Residue_perc', 'Stubble_use', 'Acre', 'Yield', 'Set',
       'Yield_per_Acre'],
      dtype='object')

In [233]:
px.scatter(df, x="Acre", y="Yield",width=900,height=550, color="District")

In [234]:
px.scatter(df, x="Acre", y="Yield",width=900,height=550, color="Block")

In [235]:
px.scatter(df, x="Acre", y="Yield",width=900,height=550, color="TransplantingIrrigationSource")

# T-tests & ANOVAs

In [236]:
# Only on train set
train = df.loc[df["Set"]=="train"]

#### T-tests / MWU tests

In [237]:
# 1- Harv_method

group1 = train[train['Harv_method']=="hand"]
print("Group 1 n=",group1.shape[0])
group2 = train[train['Harv_method']=="machine"]
print("Group 2 n=",group2.shape[0])

pg.ttest(group1["Yield"], group2["Yield"], correction=False)

Group 1 n= 3642
Group 2 n= 228


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.275205,3868,two-sided,0.00002,"[-276.91, -102.78]",0.29186,599.055,0.989671


In [238]:
pg.ttest(group1["Yield_per_Acre"], group2["Yield_per_Acre"], correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.877595,3868,two-sided,0.380218,"[-91.88, 240.8]",0.059912,0.111,0.14177


In [239]:
# MWU
pg.mwu(group1["Yield"], group2["Yield"], alternative='two-sided')

# 'RBC' : rank-biserial correlation
# 'CLES' : common language effect size

,U-val,alternative,p-val,RBC,CLES
MWU,286969.0,two-sided,4.540835e-15,0.308822,0.345589


In [240]:
pg.mwu(group1["Yield_per_Acre"], group2["Yield_per_Acre"], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,429672.5,two-sided,0.375936,-0.034887,0.517443


In [241]:
# 2- Threshing_method

group1 = train[train['Threshing_method']=="hand"]
print("Group 1 n=",group1.shape[0])
group2 = train[train['Threshing_method']=="machine"]
print("Group 2 n=", group2.shape[0])

pg.ttest(group1["Yield"], group2["Yield"], correction=False)

Group 1 n= 1772
Group 2 n= 2098


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.242037,3868,two-sided,0.025015,"[-88.35, -5.92]",0.072338,0.444,0.610856


In [242]:
pg.ttest(group1["Yield_per_Acre"], group2["Yield_per_Acre"], correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.986413,3868,two-sided,0.000068,"[-237.99, -81.07]",0.128618,97.938,0.97859


In [243]:
# MWU
pg.mwu(group1["Yield"], group2["Yield"], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,1631694.5,two-sided,5.278080e-11,0.122192,0.438904


In [244]:
pg.mwu(group1["Yield_per_Acre"], group2["Yield_per_Acre"], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,1954820.5,two-sided,0.005549,-0.051641,0.525821


In [245]:
# 3- Stubble_use

group1 = train[train['Stubble_use']=="plowed_in_soil"]
print("Group 1 n=",group1.shape[0])
group2 = train[train['Stubble_use']=="burned"]
print("Group 2 n=",group2.shape[0])

pg.ttest(group1["Yield"], group2["Yield"], correction=False)

Group 1 n= 3846
Group 2 n= 24


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.808911,3868,two-sided,0.070542,"[-503.03, 20.24]",0.370393,0.916,0.439872


In [246]:
pg.ttest(group1["Yield_per_Acre"], group2["Yield_per_Acre"], correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.781496,3868,two-sided,0.07491,"[-45.56, 951.95]",0.364779,0.876,0.429096


In [247]:
# MWU
pg.mwu(group1["Yield"], group2["Yield"], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,24109.5,two-sided,0.000053,0.477607,0.261197


In [248]:
pg.mwu(group1["Yield_per_Acre"], group2["Yield_per_Acre"], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,74720.5,two-sided,1.623552e-07,-0.619009,0.809504


#### 2. One-way ANOVAs

**a. District**

In [249]:
# Checking assumptions...
pg.homoscedasticity(train, group='District', dv='Yield_per_Acre')

,W,pval,equal_var
levene,8.690978,0.00001,False


In [250]:
pg.normality(train, group='District', dv='Yield_per_Acre')

,W,pval,normal
District,,,
Nalanda,0.424405,0.000000e+00,False
Gaya,0.837680,1.112819e-23,False
Vaishali,0.687927,4.418901e-39,False
Jamui,0.200548,0.000000e+00,False


In [251]:
train.groupby('District')['Yield'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
District,,,,
Gaya,571.16,480.0,344.00,571
Jamui,730.27,450.0,966.98,1126
Nalanda,677.20,600.0,475.84,1193
Vaishali,350.52,250.0,413.60,980


In [252]:
train.groupby('District')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
District,,,,
Gaya,2071.66,2160.0,314.01,571
Jamui,2056.61,1760.0,1855.90,1126
Nalanda,2053.43,1920.0,1007.73,1193
Vaishali,1700.26,1760.0,869.27,980


In [253]:
train.groupby('District')['Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
District,,,,
Gaya,0.27,0.22,0.17,571
Jamui,0.34,0.23,0.22,1126
Nalanda,0.33,0.31,0.18,1193
Vaishali,0.20,0.14,0.21,980


In [254]:
train.groupby('Block')['Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
Block,,,,
Chehrakala,0.18,0.18,0.09,239
Garoul,0.48,0.23,0.40,134
Gurua,0.29,0.30,0.16,358
Jamui,0.31,0.23,0.19,626
Khaira,0.38,0.27,0.25,500
Mahua,0.15,0.14,0.09,607
Noorsarai,0.35,0.31,0.19,343
Rajgir,0.33,0.31,0.18,850
Wazirganj,0.24,0.19,0.17,213


In [255]:
pg.welch_anova(data=train, dv='Yield', between='District').round(3)

,Source,ddof1,ddof2,F,p-unc,np2
0,District,3,1985.43,117.629,0.0,0.053


In [256]:
#aov = pg.anova(dv='Yield_per_Acre', between='District', subject=, data=train, detailed=True)
#aov.round(3)

pg.welch_anova(data=train, dv='Yield_per_Acre', between='District').round(3)

,Source,ddof1,ddof2,F,p-unc,np2
0,District,3,2109.545,49.741,0.0,0.016


In [257]:
# Pairwise
pg.pairwise_gameshowell(data=df, dv='Yield', between='District')

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,Gaya,Jamui,571.161121,730.269094,-159.107973,32.212647,-4.939301,1564.294771,5.172173e-06,-0.195702
1,Gaya,Nalanda,571.161121,677.198659,-106.037538,19.925691,-5.321649,1493.212429,7.085409e-07,-0.242237
2,Gaya,Vaishali,571.161121,350.518367,220.642753,19.539682,11.292034,1369.084357,0.000000e+00,0.566292
3,Jamui,Nalanda,730.269094,677.198659,53.070435,31.940655,1.661532,1618.227313,3.445663e-01,0.070241
4,Jamui,Vaishali,730.269094,350.518367,379.750727,31.701284,11.979033,1568.053776,0.000000e+00,0.498647
5,Nalanda,Vaishali,677.198659,350.518367,326.680291,19.087953,17.114475,2164.073885,0.000000e+00,0.727577


In [258]:
pg.pairwise_gameshowell(data=df, dv='Yield_per_Acre', between='District')

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,Gaya,Jamui,2071.656946,2056.605498,15.051447,56.847206,0.264770,1247.752919,9.935023e-01,0.009879
1,Gaya,Nalanda,2071.656946,2053.429941,18.227004,31.998512,0.569620,1588.012027,9.411126e-01,0.021488
2,Gaya,Vaishali,2071.656946,1700.257430,371.399515,30.720105,12.089787,1350.288690,6.261658e-14,0.517859
3,Jamui,Nalanda,2056.605498,2053.429941,3.175557,62.531178,0.050784,1713.040122,9.999530e-01,0.002143
4,Jamui,Vaishali,2056.605498,1700.257430,356.348068,61.886738,5.758068,1643.618846,6.071111e-08,0.240532
5,Nalanda,Vaishali,2053.429941,1700.257430,353.172511,40.277391,8.768505,2165.816225,0.000000e+00,0.372497


In [259]:
# Kruskal-Wallis H-test (non-parametric alternative for anova)
pg.kruskal(data=train, dv='Yield', between='District').round(4)

,Source,ddof1,H,p-unc
Kruskal,District,3,785.3467,0.0


In [260]:
# post-hoc
data = [train[train['District']=="Gaya"]['Yield_per_Acre'], #1
        train[train['District']=="Jamui"]['Yield_per_Acre'], #2
        train[train['District']=="Nalanda"]['Yield_per_Acre'], #3
        train[train['District']=="Vaishali"]['Yield_per_Acre'], #4
        ]

p_values= sp.posthoc_dunn(data, p_adjust = 'holm')
p_values.round(5)

# the only non-significant is between Vaishali and Jamui

,1,2,3,4
1,1.0,0.00000,0.0,0.00000
2,0.0,1.00000,0.0,0.31881
3,0.0,0.00000,1.0,0.00000
4,0.0,0.31881,0.0,1.00000


**b. Block**

In [261]:
# Checking assumptions...
pg.homoscedasticity(train, group='Block', dv='Yield_per_Acre')

,W,pval,equal_var
levene,8.55481,1.290157e-11,False


In [262]:
pg.normality(train, group='Block', dv='Yield_per_Acre')

,W,pval,normal
Block,,,
Noorsarai,0.559002,1.652686e-28,False
Rajgir,0.412535,0.000000e+00,False
Gurua,0.922134,1.102509e-12,False
Garoul,0.851483,2.737867e-10,False
Khaira,0.927281,7.626099e-15,False
Jamui,0.192438,1.401298e-45,False
Mahua,0.678971,2.569052e-32,False
Chehrakala,0.956465,1.269075e-06,False
Wazirganj,0.538695,2.216143e-23,False


In [263]:
train.groupby('Block')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
Block,,,,
Chehrakala,1632.47,1650.00,313.21,239
Garoul,1807.06,1870.00,303.48,134
Gurua,2042.02,2106.00,335.56,358
Jamui,2098.66,1760.00,2469.63,626
Khaira,2003.96,1980.00,348.11,500
Mahua,1703.37,1833.33,1075.93,607
Noorsarai,1989.87,1920.00,634.26,343
Rajgir,2079.08,1920.00,1123.14,850
Wazirganj,2121.46,2160.00,267.42,213


In [264]:
# Kruskal-Wallis H-test
pg.kruskal(data=train, dv='Yield_per_Acre', between='Block')#.round(3)

,Source,ddof1,H,p-unc
Kruskal,Block,8,540.81443,1.220027e-111


In [265]:
# Posthoc test - Dunn's
data = train.groupby("Block")[["Block", "Yield_per_Acre"]].apply(lambda x: x)
p_values = sp.posthoc_dunn(data, group_col="Block", val_col="Yield_per_Acre", p_adjust='holm')
p_values.round(5)

<ipython-input-265-3476d0ced939>:2: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



,Chehrakala,Garoul,Gurua,Jamui,Khaira,Mahua,Noorsarai,Rajgir,Wazirganj
Chehrakala,1.00000,0.00014,0.00000,0.23512,0.00000,0.00000,0.00000,0.00000,0.00000
Garoul,0.00014,1.00000,0.00000,0.00755,0.00002,0.32974,0.00755,0.00008,0.00000
Gurua,0.00000,0.00000,1.00000,0.00000,0.11570,0.00000,0.00207,0.00823,0.00775
Jamui,0.23512,0.00755,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Khaira,0.00000,0.00002,0.11570,0.00000,1.00000,0.00001,0.32974,0.45791,0.00000
Mahua,0.00000,0.32974,0.00000,0.00000,0.00001,1.00000,0.04531,0.00001,0.00000
Noorsarai,0.00000,0.00755,0.00207,0.00000,0.32974,0.04531,1.00000,0.45791,0.00000
Rajgir,0.00000,0.00008,0.00823,0.00000,0.45791,0.00001,0.45791,1.00000,0.00000
Wazirganj,0.00000,0.00000,0.00775,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000


**c. CropEstMethod**

In [266]:
pg.homoscedasticity(train, group='CropEstMethod', dv='Yield_per_Acre')

,W,pval,equal_var
levene,1.553415,0.198619,True


In [267]:
pg.normality(train, group='CropEstMethod', dv='Yield_per_Acre')

,W,pval,normal
CropEstMethod,,,
Manual_PuddledRandom,0.330212,0.000000e+00,False
Manual_PuddledLine,0.352747,4.258666e-28,False
Broadcasting,0.764180,3.216332e-10,False
LineSowingAfterTillage,0.896615,9.852372e-11,False


In [268]:
train.groupby('CropEstMethod')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
CropEstMethod,,,,
Broadcasting,2364.62,2560.0,314.49,83
LineSowingAfterTillage,1651.89,1664.0,411.73,206
Manual_PuddledLine,2042.47,1980.0,1014.73,235
Manual_PuddledRandom,1971.94,1890.0,1300.39,3346


In [269]:
# Kruskal-Wallis H-test
pg.kruskal(data=train, dv='Yield_per_Acre', between='CropEstMethod')#.round(3)

,Source,ddof1,H,p-unc
Kruskal,CropEstMethod,3,192.477782,1.779713e-41


In [270]:
# Posthoc test - Dunn's
data = train.groupby("CropEstMethod")[["CropEstMethod", "Yield_per_Acre"]].apply(lambda x: x)
p_values = sp.posthoc_dunn(data, group_col="CropEstMethod", val_col="Yield_per_Acre", p_adjust='holm')
p_values.round(5)

<ipython-input-270-4a056496575f>:2: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



,Broadcasting,LineSowingAfterTillage,Manual_PuddledLine,Manual_PuddledRandom
Broadcasting,1.0,0.0,0.00000,0.00000
LineSowingAfterTillage,0.0,1.0,0.00000,0.00000
Manual_PuddledLine,0.0,0.0,1.00000,0.00122
Manual_PuddledRandom,0.0,0.0,0.00122,1.00000


**d. TransplantingIrrigationSource**

In [271]:
pg.homoscedasticity(train, group='TransplantingIrrigationSource', dv='Yield_per_Acre')

,W,pval,equal_var
levene,3.558269,0.003268,False


In [272]:
pg.normality(train, group='TransplantingIrrigationSource', dv='Yield_per_Acre')

/usr/local/lib/python3.10/dist-packages/pingouin/distribution.py:242: UserWarning:

Group Well has less than 4 valid samples. Returning NaN.



,W,pval,normal
TransplantingIrrigationSource,,,
Boring,0.347204,0.000000e+00,False
Canal,0.918995,2.696044e-13,False
Rainfed,0.276310,3.034368e-20,False
Pond,0.961074,2.319658e-01,True
TubeWell,0.675438,4.937441e-04,False
Well,NaN,NaN,False


In [273]:
train.groupby('TransplantingIrrigationSource')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
TransplantingIrrigationSource,,,,
Boring,1978.38,1892.00,1328.55,3228
Canal,1880.18,1833.33,284.67,373
Pond,2008.56,1936.00,331.53,36
Rainfed,2085.01,2133.00,1456.55,103
TubeWell,1615.78,1320.00,649.34,12
Well,1744.58,1760.00,457.07,3


In [274]:
# Kruskal-Wallis H-test
pg.kruskal(data=train, dv='Yield_per_Acre', between='TransplantingIrrigationSource')#.round(3)

,Source,ddof1,H,p-unc
Kruskal,TransplantingIrrigationSource,5,18.208111,0.002697


In [275]:
# Posthoc test - Dunn's
data = train.groupby("TransplantingIrrigationSource")[["TransplantingIrrigationSource", "Yield_per_Acre"]].apply(lambda x: x)
p_values = sp.posthoc_dunn(data, group_col="TransplantingIrrigationSource", val_col="Yield_per_Acre", p_adjust='holm')
p_values.round(5)

<ipython-input-275-ea43f949f2c7>:2: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



,Boring,Canal,Pond,Rainfed,TubeWell,Well
Boring,1.00000,0.41059,1.00000,0.35888,0.12103,1.0
Canal,0.41059,1.00000,0.41059,0.05356,0.33474,1.0
Pond,1.00000,0.41059,1.00000,1.00000,0.04777,1.0
Rainfed,0.35888,0.05356,1.00000,1.00000,0.02640,1.0
TubeWell,0.12103,0.33474,0.04777,0.02640,1.00000,1.0
Well,1.00000,1.00000,1.00000,1.00000,1.00000,1.0


**e. TransplantingIrrigationPowerSource**

In [276]:
train.groupby('TransplantingIrrigationPowerSource')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
TransplantingIrrigationPowerSource,,,,
Diesel,1771.58,1980.00,1003.09,720
Electric,2031.11,1877.33,1369.02,2645
Solar,1980.00,1980.00,155.56,2


In [277]:
# Kruskal-Wallis H-test
pg.kruskal(data=train, dv='Yield_per_Acre', between='TransplantingIrrigationPowerSource')#.round(3)

,Source,ddof1,H,p-unc
Kruskal,TransplantingIrrigationPowerSource,2,0.383014,0.825714


**f. PCropSolidOrgFertAppMethod**

In [278]:
print(pg.homoscedasticity(train, group='PCropSolidOrgFertAppMethod', dv='Yield_per_Acre'))
print(pg.normality(train, group='PCropSolidOrgFertAppMethod', dv='Yield_per_Acre'))

                W          pval  equal_var
levene  14.694136  1.738101e-09      False
                                   W     pval normal
PCropSolidOrgFertAppMethod                          
SoilApplied                 0.684401  0.00000  False
Broadcasting                0.341184  0.00000  False
RootApplication             0.849499  0.07358   True
Spray                            NaN      NaN  False


/usr/local/lib/python3.10/dist-packages/pingouin/distribution.py:242: UserWarning:

Group Spray has less than 4 valid samples. Returning NaN.



In [279]:
train.groupby('PCropSolidOrgFertAppMethod')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
PCropSolidOrgFertAppMethod,,,,
Broadcasting,1898.62,1760.00,2255.74,841
RootApplication,1932.22,2200.00,557.59,9
SoilApplied,2055.57,2055.24,581.99,1680
Spray,1186.33,1755.00,989.29,3


In [280]:
pg.kruskal(data=train, dv='Yield_per_Acre', between='PCropSolidOrgFertAppMethod')#.round(3)

,Source,ddof1,H,p-unc
Kruskal,PCropSolidOrgFertAppMethod,3,260.774627,3.056595e-56


In [281]:
data = train.groupby("PCropSolidOrgFertAppMethod")[["PCropSolidOrgFertAppMethod", "Yield_per_Acre"]].apply(lambda x: x)
p_values = sp.posthoc_dunn(data, group_col="PCropSolidOrgFertAppMethod", val_col="Yield_per_Acre", p_adjust='holm')
p_values.round(5)

<ipython-input-281-96f8e64f86f9>:1: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



,Broadcasting,RootApplication,SoilApplied,Spray
Broadcasting,1.00000,0.28819,0.00000,0.42464
RootApplication,0.28819,1.00000,0.72209,0.22014
SoilApplied,0.00000,0.72209,1.00000,0.07778
Spray,0.42464,0.22014,0.07778,1.00000


**g. MineralFertAppMethod**

In [282]:
print(pg.homoscedasticity(train, group='MineralFertAppMethod.1', dv='Yield_per_Acre'))
print(pg.normality(train, group='MineralFertAppMethod.1', dv='Yield_per_Acre'))

               W      pval  equal_var
levene  0.614476  0.540985       True
                               W          pval  normal
MineralFertAppMethod.1                                
Broadcasting            0.343002  0.000000e+00   False
RootApplication         0.286458  3.222472e-12   False
SoilApplied             0.949445  1.074642e-02   False


In [283]:
train.groupby('MineralFertAppMethod.1')['Yield_per_Acre'].agg(['mean', 'median', 'std', 'count']).round(2)

,mean,median,std,count
MineralFertAppMethod.1,,,,
Broadcasting,1970.31,1907.45,1315.58,3288
RootApplication,2103.95,1706.67,2382.09,37
SoilApplied,2159.83,2200.00,466.72,64


In [284]:
pg.kruskal(data=train, dv='Yield_per_Acre', between='MineralFertAppMethod.1')#.round(3)

,Source,ddof1,H,p-unc
Kruskal,MineralFertAppMethod.1,2,27.776811,9.296985e-07


In [285]:
data = train.groupby("MineralFertAppMethod.1")[["MineralFertAppMethod.1", "Yield_per_Acre"]].apply(lambda x: x)
p_values = sp.posthoc_dunn(data, group_col="MineralFertAppMethod.1", val_col="Yield_per_Acre", p_adjust='holm')
p_values.round(5)

<ipython-input-285-162c7abfa587>:1: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



,Broadcasting,RootApplication,SoilApplied
Broadcasting,1.00000,0.00324,0.00003
RootApplication,0.00324,1.00000,0.00000
SoilApplied,0.00003,0.00000,1.00000


In [286]:
# Do farmers typically use the same method for 1st and 2nd fertilizer doses?
same_value_count = (train['MineralFertAppMethod'] == train['MineralFertAppMethod.1']).sum()
print(same_value_count)
print(same_value_count / train.shape[0] * 100)

2856
73.7984496124031


In [287]:
train.loc[train['MineralFertAppMethod'] == train['MineralFertAppMethod.1']]["MineralFertAppMethod.1"].value_counts()

Broadcasting       2797
SoilApplied          56
RootApplication       3
Name: MineralFertAppMethod.1, dtype: int64

In [288]:
train.loc[train['MineralFertAppMethod'] != train['MineralFertAppMethod.1']]["MineralFertAppMethod.1"].value_counts()

Broadcasting       491
RootApplication     34
SoilApplied          8
Name: MineralFertAppMethod.1, dtype: int64

In [289]:
print(train["MineralFertAppMethod"].isnull().sum())
print(train["MineralFertAppMethod.1"].isnull().sum())

0
481
